<h1>OCR model</h1>

In [2]:
!apt-get install -y poppler-utils
!wget https://github.com/tesseract-ocr/tessdata/raw/main/vie.traineddata


Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libpoppler97 poppler-data
Suggested packages:
  ghostscript fonts-japanese-mincho | fonts-ipafont-mincho
  fonts-japanese-gothic | fonts-ipafont-gothic fonts-arphic-ukai
  fonts-arphic-uming fonts-nanum
The following NEW packages will be installed:
  libpoppler97 poppler-data poppler-utils
0 upgraded, 3 newly installed, 0 to remove and 69 not upgraded.
Need to get 2564 kB of archives.
After this operation, 16.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/main amd64 poppler-data all 0.4.9-2 [1475 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libpoppler97 amd64 0.86.1-0ubuntu1.4 [916 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 poppler-utils amd64 0.86.1-0ubuntu1.4 [174 kB]
Fetched 2564 kB in 2s (1112 kB/s)       
Selecting previously unselected package

# Create traineddata 


In [3]:
!mkdir -p /kaggle/working/tesseract-ocr/4.00/tessdata/
!mv vie.traineddata /kaggle/working/tesseract-ocr/4.00/tessdata/
import os
os.environ['TESSDATA_PREFIX'] = '/kaggle/working/tesseract-ocr/4.00/tessdata/'


# Import 


In [4]:
import pytesseract
from PIL import Image
import os
from pdf2image import convert_from_path
import os
import cv2
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed, ProcessPoolExecutor
import matplotlib.pyplot as plt
import numpy as np
from multiprocessing import Pool, cpu_count



# Convert PDF to Img

In [9]:
def convert_and_save_page(args):
    page_number, pdf_path, output_folder, dpi = args
    images = convert_from_path(pdf_path, dpi=dpi, first_page=page_number, last_page=page_number)
    image_path = os.path.join(output_folder, f'page{page_number - 1}.jpg')
    images[0].save(image_path, 'JPEG')

def main(pdf_path, output_folder, dpi):
    os.makedirs(output_folder, exist_ok=True)

    num_pages = len(convert_from_path(pdf_path, dpi=10))

    tasks = [(page_number, pdf_path, output_folder, dpi) for page_number in range(1, num_pages + 1)]

    with Pool(cpu_count()) as pool:
        list(tqdm(pool.imap(convert_and_save_page, tasks), total=num_pages))

if __name__ == "__main__":
    pdf_path = '/kaggle/input/pdf-dataset-ocr/Do_Kien_Lam_29_01_2003_TSH.pdf'
    output_folder = '/kaggle/working/img/'
    dpi = 800
    main(pdf_path, output_folder, dpi)


100%|██████████| 55/55 [01:43<00:00,  1.89s/it]

All pages have been converted and saved.


100%|██████████| 55/55 [01:43<00:00,  1.89s/it]


# Best Model 1

In [ ]:
def preprocess_image(image_path):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)
    _, binary_image = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    binary_image = cv2.morphologyEx(binary_image, cv2.MORPH_CLOSE, kernel)
    return binary_image

def ocr_image(image_path):
    processed_image = preprocess_image(image_path)
    pil_image = Image.fromarray(processed_image)
    text = pytesseract.image_to_string(pil_image, config='--psm 6', lang='vie')
    return (image_path, text)

def main(folder_path):
    image_paths = [os.path.join(folder_path, name) for name in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, name))]
    
    with Pool(cpu_count()) as p:
        results = list(tqdm(p.imap(ocr_image, image_paths), total=len(image_paths), desc='Processing images'))
    
    for image_path, text in results:
        original_image = cv2.imread(image_path)

if __name__ == "__main__":
    path = '/kaggle/working/img'
    main(path)


Processing images: 100%|██████████| 55/55 [06:55<00:00,  7.55s/it]